In [1]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score, roc_auc_score, roc_curve, auc
from scipy.stats import sem
from capymoa.anomaly import HalfSpaceTrees, Autoencoder, OnlineIsolationForest, StreamRHF
from capymoa.datasets import Electricity, Hyper100k, CovtypeNorm, RTG_2abrupt
from capymoa.evaluation import AnomalyDetectionEvaluator

In [2]:
# List of datasets and models
#datasets = [Electricity, Hyper100k, CovtypeNorm, RTG_2abrupt]
#datasets = [Electricity, Hyper100k]
datasets = [Hyper100k]
models = {
#    "HalfSpaceTrees": HalfSpaceTrees,
#    "Autoencoder": Autoencoder,
#    "OnlineIsolationForest": OnlineIsolationForest,
    "StreamRHF": StreamRHF
}

# Directory for saving results
results_dir = os.path.join(os.getcwd(), "results")
os.makedirs(results_dir, exist_ok=True)

# Results storage
all_results = []

In [4]:
# Main loop over datasets and models
for dataset_cls in datasets:
    dataset_name = dataset_cls.__name__
    print(f"\nDataset: {dataset_name}")
    stream = dataset_cls()
    schema = stream.get_schema()

    # Load dataset labels
    labels = []

    processed_instances = 0
    #while processed_instances < 10:
    #    processed_instances += 1

    while processed_instances < 20000 and stream.has_more_instances():
        instance = stream.next_instance()
        labels.append(instance.y_index)
        processed_instances += 1
    
    #while stream.has_more_instances():
    #    instance = stream.next_instance()
    #    labels.append(instance.y_index)
    processed_instances = 0 
    labels = np.array(labels)
    stream.restart()

    for model_name, ModelClass in models.items():
        print(f"\nRunning Model: {model_name}")
        model_results = []
        ap_scores, auc_scores, auc_paper_scores, auc_capymoa_scores, execution_times = [], [], [], [], []

        for run in range(1):  #For now, let's just try one run per dataset and model since they are big
            print(f"Run {run + 1}")
            learner = ModelClass(schema)
            evaluator = AnomalyDetectionEvaluator(schema)

            stream.restart()
            anomaly_scores = []
            start_time = time.time()

            #processed_instances = 0
            #while processed_instances < 10:
            #    processed_instances += 1
            #while stream.has_more_instances():
            #    instance = stream.next_instance()
            #    proba = learner.score_instance(instance)
            #    anomaly_scores.append(1 - proba)  # Flip score for consistency
            #    evaluator.update(instance.y_index, proba)
            #    learner.train(instance)

            processed_instances = 0

            while processed_instances < 20000 and stream.has_more_instances():
                instance = stream.next_instance()
                proba = learner.score_instance(instance)
                anomaly_scores.append(1 - proba)  # Flip score for consistency
                evaluator.update(instance.y_index, proba)
                learner.train(instance)
                processed_instances += 1

            # Compute metrics
            auc_score_capymoa = evaluator.auc()
            anomaly_scores = np.array(anomaly_scores)
            ap_score = average_precision_score(labels, anomaly_scores)
            auc_score = roc_auc_score(labels, anomaly_scores)
            fpr, tpr, _ = roc_curve(labels, anomaly_scores)
            auc_paper = auc(fpr, tpr)
            execution_time = time.time() - start_time

            # Save run results
            run_result = {
                'Dataset': dataset_name,
                'Model': model_name,
                'Run': run + 1,
                'AP': ap_score,
                'AUC_capymoa': auc_score_capymoa,
                'AUC (sklearn)': auc_score,
                'AUC (paper)': auc_paper,
                'Execution Time (s)': execution_time
            }
            model_results.append(run_result)
            all_results.append(run_result)

            # Append metrics for summary
            ap_scores.append(ap_score)
            auc_scores.append(auc_score)
            auc_paper_scores.append(auc_paper)
            auc_capymoa_scores.append(auc_score_capymoa)
            execution_times.append(execution_time)

        # Save checkpoint when using various models and datsets
        #results_df = pd.DataFrame(all_results)
        #checkpoint_path = os.path.join(results_dir, "all_run_results_checkpoint.csv")
        #results_df.to_csv(checkpoint_path, index=False)
        #print(f"Checkpoint saved for model {model_name} at {checkpoint_path}")
        # Save checkpoint for only one endpoint
        results_df = pd.DataFrame(all_results)
        checkpoint_path = os.path.join(results_dir, "streamrhf_hyper100k_results_checkpoint.csv")
        results_df.to_csv(checkpoint_path, index=False)
        print(f"Checkpoint saved for model {model_name} at {checkpoint_path}")

        # Summarize results
        metrics = {
            'AP': ap_scores,
            'AUC (sklearn)': auc_scores,
            'AUC (paper)': auc_paper_scores,
            'AUC (CapyMOA)': auc_capymoa_scores,
            'Execution Time': execution_times
        }
        summary = {metric: (np.mean(scores), sem(scores) * 1.96) for metric, scores in metrics.items()}
        
        # Save summary
        summary_data = {
            'Metric': list(summary.keys()),
            'Mean': [mean for mean, _ in summary.values()],
            '95% CI': [ci for _, ci in summary.values()]
        }
        summary_df = pd.DataFrame(summary_data)
        model_folder = os.path.join(results_dir, model_name)
        os.makedirs(model_folder, exist_ok=True)
        summary_path = os.path.join(model_folder, f"{dataset_name}_summary.csv")
        summary_df.to_csv(summary_path, index=False)
        print(f"Summary saved for {model_name} at {summary_path}")


Dataset: Hyper100k

Running Model: StreamRHF
Run 1
our StreamRHF initialized
20
Checkpoint saved for model StreamRHF at /home/infres/cchavez-23/Stream-Random-Histogram-Forest/src/capymoa/anomaly/tests capymoa datasets/results/streamrhf_hyper100k_results_checkpoint.csv
Summary saved for StreamRHF at /home/infres/cchavez-23/Stream-Random-Histogram-Forest/src/capymoa/anomaly/tests capymoa datasets/results/StreamRHF/Hyper100k_summary.csv


/tmp/ipykernel_2394272/3658170013.py:110: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  summary = {metric: (np.mean(scores), sem(scores) * 1.96) for metric, scores in metrics.items()}
